## WF2 Module 2

We want to find molecular phenotypes that are integral to a disease, and then chemicals that activate those phenotypes, and then genes that clean up those bad chemicals.   The tricky part with current data is that it's difficult to make the jump to chemicals.

For instance, if we say that molecular phenotype is a GO process, we might find that e.g. various DNA repair processes are involved with Fanconi Anemia.  But the only chemicals that are linked to these processes are participants in the process, like DNA.

From the chemical side, there are properties on chemicals in CHEBI, such as being genotoxic.  However, there's not any semantic value to these properties because they are just strings with no relation to e.g. GO terms.  This gap is being filled by CHIRO, but that is still ongoing.

To leapfrog this, we'll try (1) grabbing chemicals that look bad for the disease, (2) try to do some enrichments on their properties to get other similar chemicals, and (3) look for genes that degrade those things.

In [1]:
#To make nicer looking outputs
from IPython.core.display import display, HTML
import requests
import pandas as pd
import os
import sys

#Load some functions for parsing quick output
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
        sys.path.append(module_path)
from gg_functions import parse_answer, get_view_url, expand, quick

robokop='robokop.renci.org' 

In [2]:
disease = "MONDO:0019391"  #Fanconi Anemia

In [3]:
bad_chemicals = expand('disease',disease,'chemical_substance',predicate='contributes_to',direction='in')

In [4]:
df = parse_answer(bad_chemicals,node_list=['n1'])

In [5]:
df

,n1 - id,n1 - name,score
0,CHEBI:23704,diepoxybutane,0.513305
1,CHEBI:16856,glutathione,0.508893
2,CHEBI:28901,busulfan,0.360132


What properties do these things have in common?

In [6]:
chems = [x for x in bad_chemicals['knowledge_graph']['nodes'] if 'chemical_substance' in x['type']]
from collections import defaultdict
prop_counts = defaultdict(int)
for n in chems:
    for p in n:
        prop_counts[p] += 1
items = list(prop_counts.items())
count_frame = pd.DataFrame.from_records(items,columns=['property','count_in_subset'])
count_frame.head()

,property,count_in_subset
0,eukaryotic_metabolite,1
1,therapeutic_flag,3
2,topical,3
3,application,2
4,mass,3


Many of these propertries are assigned to many chemicals (like SMILES or name).  We want to find the ones that are overrepresented.  To do that we need to know the details of how often each property is used in the database.  At the moment, ROBOKOP does not expose functions to allow this, but we have pre-calculated such a list:

In [7]:
property_counts = pd.read_csv('../../examples/chemprops.txt','\t')
property_counts.head()

,property,count
0,appetite_enhancer,2
1,Wiskott_Aldrich_syndrome_protein_inhibitor,1
2,peptide_coupling_reagent,2
3,EC_3.4.22.52_(calpain_1)_inhibitor,3
4,EC_2.3.1.5_(arylamine_N_acetyltransferase)_inh...,1


In [8]:
df = pd.merge(count_frame, property_counts,on='property',how='inner')

df.head()

,property,count_in_subset,count
0,eukaryotic_metabolite,1,11711
1,therapeutic_flag,3,244803
2,topical,3,244803
3,application,2,8162
4,mass,3,97320


In [9]:
from scipy.stats import hypergeom
#in scipy:
#The hypergeometric distribution models drawing objects from a bin. 
#M is the total number of objects, 
#n is total number of Type I objects. 
#The random variate represents the number of Type I objects in N drawn without replacement from the total population.

total_chemical_count=355413 # M above
subset_count = len(chems) # N above

def calc_enrich_p(x,n):
    return hypergeom.sf(x-1, total_chemical_count, n, subset_count)

In [10]:
df['enrichment_p'] = df.apply(lambda x: calc_enrich_p(x['count_in_subset'], x['count']), axis=1)
df.sort_values(by='enrichment_p')

,property,count_in_subset,count,enrichment_p
44,mutagen,2,195,8.981212e-07
42,genotoxin,2,237,1.327776e-06
45,aetiopathogenetic_role,2,835,1.651312e-05
47,insect_sterilant,1,7,5.908519e-05
51,chemosterilant,1,7,5.908519e-05
13,skin_lightening_agent,1,7,5.908519e-05
18,cosmetic,1,12,1.012875e-04
56,teratogenic_agent,1,47,3.966702e-04
39,biological_role,3,27273,4.518088e-04
52,alkylating_agent,1,78,6.582463e-04


Now we can see that the most enriched property is "mutagen", which occurs in 2 of the 3 chemicals that are contributors to FA, but in only 195 out of the 355k chemicals.  Let's look for genes that degrade mutagens.  We're going to lose the connection to FA, so that won't be included in our score, which is unfortunate.

In [11]:
def create_question(disease_id,prop):
    return {
    "machine_question": {
        "nodes": [
            {
                "id": "n0",
                "type": "chemical_substance",
                prop: True
            },
            {
                "id": "n1",
                "type": "gene"
            },
            {
                "id": "n2",
                "type": "disease",
                "curie": disease_id
            }
        ],
        "edges": [
            {
                "id": "e0",
                "source_id": "n1",
                "target_id": "n0",
                "type": "increases_degradation_of"
            }
        ]
    }
}

In [12]:
q = create_question('MONDO:0019391','mutagen')
a = quick(q)

Return Status: 200


In [13]:
aframe = parse_answer(a,node_list=['n0','n1'])

In [14]:
aframe

,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:15343,acetaldehyde,HGNC:404,ALDH2,0.263911
1,CHEBI:15343,acetaldehyde,HGNC:405,ALDH3A1,0.257250
2,CHEBI:27732,caffeine,HGNC:2596,CYP1A2,0.190395
3,CHEBI:4027,cyclophosphamide,HGNC:2637,CYP3A4,0.183836
4,CHEBI:17754,glycerol,HGNC:4289,GK,0.181862
5,CHEBI:15343,acetaldehyde,HGNC:407,ALDH1B1,0.181362
6,CHEBI:15343,acetaldehyde,HGNC:877,ALDH7A1,0.181250
7,CHEBI:15343,acetaldehyde,HGNC:409,ALDH1A3,0.181236
8,CHEBI:15343,acetaldehyde,HGNC:412,ALDH9A1,0.181208
9,CHEBI:15343,acetaldehyde,HGNC:410,ALDH3B1,0.181194


This finds the expected result that ALDH2 is a potential modifier for FA.   Potentially, but looking for things that degrade into acetaldehyde or other chemicals in n0 we could find a list of environemntal exposures to avoid.  Note that, as ever, we are at the mercy of the data - caffeine is mutagenic at high dose but mostly in non-mammalian cells.